In [1]:
!pip install langgraph langchain-core gigachat pdfplumber PyPDF2 python-dotenv

In [1]:
import os
print(os.getcwd())
print(os.listdir())


C:\Users\aleks\DS projects\pd_agent_system
['.env', '.ipynb_checkpoints', 'agent_solution.ipynb', 'resumes']


In [2]:
import os
from typing import Dict, Any, List, Optional
from dataclasses import dataclass
import pdfplumber
import json
from dotenv import load_dotenv

from langgraph.graph import StateGraph, END
from gigachat import GigaChat

load_dotenv()

# Загрузка переменных окружения
GIGACHAT_API_KEY = os.getenv("GIGACHAT_API_KEY")


In [3]:
class PROMPTS:
    ROADMAP = """Ты — опытный карьерный наставник и технический эксперт в IT-сфере.
Создай подробную дорожную карту для перехода в профессию {profession} для уровня {goal}.
Начальный уровень пользователя: {background}.
Анализируй по следующим критериям:
1. НАПРАВЛЕНИЕ И УРОВНИ
• Текущий уровень пользователя
• Целевой уровень
• Разрыв в компетенциях
2. СРОКИ ОБУЧЕНИЯ
• Реалистичный временной диапазон
• Количество часов в неделю
• Этапы с конкретными дедлайнами
3. ШАГИ ПО ОБУЧЕНИЮ (разбить по месяцам)
• Месяц 1: конкретные технологии и темы
• Месяц 2: конкретные технологии и темы
• Месяц 3: конкретные технологии и темы
• Месяц 4+: углубление и специализация
4. ЧТО КОНКРЕТНО УЧИТЬ
• Языки программирования с версиями
• Фреймворки и библиотеки
• Инструменты и ПО
• Методологии и практики
• Теория (алгоритмы, архитектура и т.д.)
5. РЕСУРСЫ ДЛЯ ОБУЧЕНИЯ
• Бесплатные: курсы, документация, YouTube-каналы
• Платные курсы (если стоит того)
• Книги с авторами и изданиями
• Практические платформы (LeetCode, Codewars, Kaggle)
• Сообщества и чаты
6. ПРАКТИЧЕСКИЕ ПРОЕКТЫ
• Проект 1: стек, функционал, срок
• Проект 2: стек, функционал, срок
• Проект 3: стек, функционал, срок
• Где размещать (GitHub, Behance и т.д.)
7. КАК ИСКАТЬ РАБОТУ
• Где искать вакансии (конкретные сайты)
• Как составить резюме под эту позицию
• Какие навыки выделять
• Как готовиться к собеседованиям
• Какие вопросы задавать работодателю
• Средние зарплаты на рынке

Для каждого пункта давай максимально конкретные названия, инструменты, технологии и сроки.
Используй реальные стеки технологий для этой профессии.
Дай практические советы из опыта найма в IT.
В конце сделай итоговую оценку сложности перехода: «Реально за 3-4 месяца», «Требует 6+ месяцев», «Сложный переход» и т.д.
"""

    # Этот промпт — для генерации резюме/шаблонов/сопроводительных (создание, не анализ)
    RESUME_AND_COVER_TEMPLATES = """Роль:
Ты карьерный консультант и эксперт по составлению резюме для IT, аналитики, менеджмента продукта и смежных областей.
Твоя задача — помогать пользователям создавать профессиональные, структурированные и эффективные резюме,
адаптированные под конкретные вакансии и компании, с учётом требований рекрутеров и систем отбора (ATS).

Основные принципы:
1) Краткость и ясность: понятно за 15–30 секунд.
2) Технологии и результат: наверху ключевые навыки, в опыте метрики и глаголы действия.
3) ATS: ключевые слова из вакансии, чистый формат, стандартные шрифты, чёткие разделы.
4) Без лишнего: фото/грейд/личные данные не добавлять (если не требуется).
5) “О себе”: 2–3 строки, без общих фраз.

Структура резюме:
• ФИО
• Контакты
• Ключевые навыки
• Опыт работы (действие — результат, STAR)
• Проекты
• Образование
• Курсы
• Языки
• О себе

Формат взаимодействия:
- Если данных мало — запроси недостающие.
- Если есть вакансия/компания — предложи адаптацию под требования и ключевые слова.
- Генерация: выдай резюме в Markdown для копирования.
- Дополнительно: сопроводительное письмо + советы по собеседованию.

Тон: дружелюбный, поддерживающий, профессиональный, без воды.
"""

    DOC_ANALYSIS = """Ты — профессиональный карьерный аналитик и эксперт по оценке резюме и сопроводительных писем.
Твоя задача — проводить глубокий, структурированный, объективный анализ и давать точные рекомендации.

Обязательно выдавай ответ в формате:
- Краткое резюме анализа (2–4 предложения)
- Сильные стороны
- Слабые стороны и проблемы
- Риски/сомнения работодателя
- Рекомендации по улучшению
- Предлагаемая улучшенная версия (если документ короткий)
- Предложения по стилю и тону

Точность:
- Если информации мало — запрашивай недостающее.
- Не выдумывай факты.

Стиль: строгий, профессиональный, без воды.
"""

    VACANCY_LEGAL_FLAGS = """Ты — квалифицированный юрист по трудовому праву с многолетним опытом, специализирующийся на проверке вакансий
на соответствие законодательству (ТК РФ), ФЗ №59-ФЗ «О запрете анонимных вакансий» и ФЗ-152 «О персональных данных».

Проанализируй вакансию и определи:
- Красные флаги (перечень из задания) + пояснения:
  почему это флаг, правовое последствие/практическая проблема, статья закона (если применимо).
- Зелёные флаги (перечень из задания) + пояснения.

Сформулируй итоговый вердикт:
- стоит ли откликаться, при каких условиях,
- какие вопросы задать на собеседовании,
- итоговая оценка: «Рекомендуется к отклику», «Требует осторожности» или «Высокий риск - не рекомендуется».

Вот вакансия:
{vacancy_text}
"""


In [4]:
 def parse_kv(text: str) -> Dict[str, str]:
    out = {}
    for line in text.splitlines():
        if ":" in line:
            k, v = line.split(":", 1)
            out[k.strip().lower()] = v.strip()
    return out

In [5]:
#Состояние графа
from dataclasses import dataclass
from typing import Dict, Any, List, Optional
import pdfplumber
from langgraph.graph import StateGraph, END

# @dataclass state ок, но langgraph часто возвращает dict — поэтому далее используем getattr/поля аккуратно
@dataclass
class AgentState:
    messages: List[Dict[str, Any]]
    current_agent: str
    user_query: str
    extracted_text: Optional[str] = None
    final_response: Optional[str] = None


#  Базовый агент
class BaseAgent:
    def __init__(self, name: str, tools: List[Any]):
        self.name = name
        self.tools = tools
        self.model = GigaChat(credentials=GIGACHAT_API_KEY, verify_ssl_certs=False)

    def chat(self, user_query: str, history: Optional[List[Dict[str, str]]] = None) -> str:
        """
        Обращение к LLM с учётом истории диалога.
        history: [{"role": "user"/"assistant", "content": "..."}]
        """
        if history:
            hist_text = ""
            for msg in history[-10:]:
                role = "Пользователь" if msg["role"] == "user" else "Ассистент"
                hist_text += f"{role}: {msg['content']}\n"

            prompt = (
                f"{hist_text}\n"
                f"Пользователь: {user_query}\n"
                f"Ассистент:"
            )
        else:
            prompt = user_query

        resp = self.model.chat(prompt)
        return resp.choices[0].message.content

    # ✅ Вынесли PDF-экстрактор в базовый класс, чтобы оба агента могли читать PDF
    def extract_pdf_text_tool(self, pdf_path: str) -> str:
        """Извлекает текст из PDF файла"""
        try:
            text = ""
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    text += (page.extract_text() or "") + "\n"
            return text[:4000]  # лимит чуть расширим
        except Exception as e:
            return f"Ошибка при чтении PDF: {str(e)}"

class OrchestratorAgent(BaseAgent):
    def __init__(self):
        super().__init__("orchestrator", [])

    def route_query(self, state: AgentState) -> str:
        """Определяет, какой агент должен обработать запрос"""
        query = state.user_query.lower()

        career_prep_keywords = [
            "резюме", "cv", "roadmap", "карьер", "сопровод",
            "письмо", "шаблон", "подготов"
        ]
        hiring_keywords = [
            "ваканси", "собесед", "интервью",
            "вопрос", "red flag", "green flag"
        ]

        prep_score = sum(1 for k in career_prep_keywords if k in query)
        hiring_score = sum(1 for k in hiring_keywords if k in query)

        if prep_score >= hiring_score:
            return "career_preparation_agent"
        else:
            return "hiring_agent"

In [6]:
class CareerPreparationAgent(BaseAgent):
    def __init__(self):
        tools = [
            self.create_roadmap_tool,
            self.analyze_resume_tool,
            self.generate_resume_or_templates_tool,
            self.generate_cover_letter_tool,
            self.extract_pdf_text_tool,
        ]
        super().__init__("career_preparation_agent", tools)

    def create_roadmap_tool(self, profession: str, goal: str, background: str, history=None) -> str:
        prompt = PROMPTS.ROADMAP.format(
            profession=profession,
            goal=goal,
            background=background
        )
        return self.chat(prompt, history=history)

    def generate_resume_or_templates_tool(self, user_data: str, vacancy_text: Optional[str] = None, history=None) -> str:
        # user_data — то, что пользователь про себя дал (навыки/опыт/проекты)
        extra = ""
        if vacancy_text:
            extra = f"\n\nВакансия/требования для адаптации (используй как источник ключевых слов для ATS):\n{vacancy_text}\n"
        prompt = f"{PROMPTS.RESUME_AND_COVER_TEMPLATES}\n\nДанные пользователя:\n{user_data}\n{extra}\n\nВыдай результат."
        return self.chat(prompt, history=history)

    def analyze_resume_tool(self, pdf_path: str, history=None) -> str:
        text = self.extract_pdf_text_tool(pdf_path)
        prompt = f"{PROMPTS.DOC_ANALYSIS}\n\nТекст резюме:\n{text}\n\nВыдай получившийся результат."
        return self.chat(prompt, history=history)

    def analyze_cover_letter_text_tool(self, cover_letter_text: str, history=None) -> str:
        prompt = f"{PROMPTS.DOC_ANALYSIS}\n\nТекст сопроводительного письма:\n{cover_letter_text}\n\nВыдай получившийся результат."
        return self.chat(prompt, history=history)

    def generate_cover_letter_tool(self, position: str, company: str, user_profile: str = "", history=None) -> str:
        # Лаконичный генератор; если хочешь — тоже можно сделать отдельным большим промптом
        prompt = f"""{PROMPTS.RESUME_AND_COVER_TEMPLATES}
        Сгенерируй сопроводительное письмо на позицию "{position}" в компанию "{company}".
        Данные пользователя (если есть):
        {user_profile}
        Формат: Markdown. Без воды. 180–260 слов.
        """
        return self.chat(prompt, history=history)


In [7]:
class HiringAgent(BaseAgent):
    def __init__(self):
        tools = [
            self.analyze_vacancy_text_tool,
            self.analyze_vacancy_pdf_tool,
            self.generate_interview_questions_tool,
            self.extract_pdf_text_tool,
        ]
        super().__init__("hiring_agent", tools)

    def analyze_vacancy_text_tool(self, vacancy_text: str, history=None) -> str:
        prompt = PROMPTS.VACANCY_LEGAL_FLAGS.format(vacancy_text=vacancy_text)
        return self.chat(prompt, history=history)

    def analyze_vacancy_pdf_tool(self, pdf_path: str, history=None) -> str:
        text = self.extract_pdf_text_tool(pdf_path)
        prompt = PROMPTS.VACANCY_LEGAL_FLAGS.format(vacancy_text=text)
        return self.chat(prompt, history=history)

    def generate_interview_questions_tool(self, position: str, level: str = "middle", history=None) -> str:
        prompt = f"""
        Сгенерируй список из 10-15 вопросов для собеседования на позицию {position} уровня {level}.
        Включи:
        - Технические вопросы
        - Поведенческие вопросы
        - Вопросы о мотивации
        - Кейсовые задания
        Для каждого вопроса укажи, на что обращать внимание в ответе.
        """
        return self.chat(prompt, history=history)


In [8]:
class MultiAgentSystem:
    def __init__(self):
        self.orchestrator = OrchestratorAgent()
        self.career_preparation_agent = CareerPreparationAgent()
        self.hiring_agent = HiringAgent()

        # ✅ Инициализация истории (у тебя её не было)
        self.chat_history: List[Dict[str, str]] = []

        self.graph = self._build_graph()
    def _last_assistant_message(self) -> str:
        for msg in reversed(self.chat_history):
            if msg["role"] == "assistant":
                return msg["content"]
        return ""

    def _append_history(self, user_query: str, response: str) -> None:
        self.chat_history.append({"role": "user", "content": user_query})
        self.chat_history.append({"role": "assistant", "content": response})

    # Построение графа
    def _build_graph(self) -> StateGraph:
        workflow = StateGraph(AgentState)

        workflow.add_node("orchestrator", self._orchestrator_node)
        workflow.add_node("career_preparation_agent", self._career_preparation_agent_node)
        workflow.add_node("hiring_agent", self._hiring_agent_node)

        workflow.set_entry_point("orchestrator")

        workflow.add_conditional_edges(
            "orchestrator",
            self._route_after_orchestrator,
            {
                "career_preparation_agent": "career_preparation_agent",
                "hiring_agent": "hiring_agent",
            },
        )

        workflow.add_edge("career_preparation_agent", END)
        workflow.add_edge("hiring_agent", END)

        return workflow.compile()

    # Узлы графа
    def _orchestrator_node(self, state: AgentState) -> AgentState:
        agent_type = self.orchestrator.route_query(state)
        state.current_agent = agent_type
        return state

    def _career_preparation_agent_node(self, state: AgentState) -> AgentState:
        response = self._process_with_agent(self.career_preparation_agent, state.user_query, self.chat_history)
        state.final_response = response
        return state

    def _hiring_agent_node(self, state: AgentState) -> AgentState:
        # ✅ было self._hiring_agent (не существует)
        response = self._process_with_agent(self.hiring_agent, state.user_query, self.chat_history)
        state.final_response = response
        return state

    def _route_after_orchestrator(self, state: AgentState) -> str:
        return state.current_agent

    # ✅ Переписано под новую архитектуру
    def _process_with_agent(self, agent: BaseAgent, query: str, history: Optional[List[Dict[str, str]]] = None) -> str:
        q = query.lower().strip()
        
        # --- CareerPreparationAgent ---
        if isinstance(agent, CareerPreparationAgent):
            # Roadmap intent
            if "roadmap" in q or "дорожн" in q or "план" in q:
                data = parse_kv(query)

                # пользователь прислал параметры в формате "ключ: значение"
                if {"profession", "goal", "background"}.issubset(data.keys()):
                    return agent.create_roadmap_tool(
                        profession=data["profession"],
                        goal=data["goal"],
                        background=data["background"],
                        history=history
                    )

                # если параметров не хватает — просим одним сообщением
                return (
                    "Чтобы составить дорожную карту, пришли 3 строки:\n"
                    "profession: ...\n"
                    "goal: ...\n"
                    "background: ...\n"
                )
            # Анализ сопроводительного (если пользователь вставил текст)
            if "сопровод" in q and ("анализ" in q or "разбор" in q or "оцен" in q):
                return (
                    "Вставь сюда текст сопроводительного письма (полностью), и я сделаю структурированный разбор."
                )

            # Генерация резюме/шаблона
            if "шаблон" in q or "резюме" in q and ("сгенер" in q or "состав" in q or "напис" in q):
                return (
                    "Чтобы сгенерировать резюме/шаблон, пришли данные одним сообщением:\n"
                    "- цель (роль + компания/вакансия если есть)\n"
                    "- hard skills\n"
                    "- опыт (период, должность, компания, достижения)\n"
                    "- проекты (что сделал, стек, результат)\n"
                    "- образование/курсы\n"
                    "- языки\n"
                    "- 2–3 строки “о себе” (если есть)\n\n"
                    "Если есть текст вакансии — добавь, адаптирую под ATS."
                )

            # По умолчанию
            return agent.chat(query, history=history)

        # --- HiringAgent ---
        if isinstance(agent, HiringAgent):
            last_assistant = self._last_assistant_message()
            waiting_for_vacancy_text = (
                "вставь сюда текст вакансии" in last_assistant.lower()
                or "пришли путь к pdf" in last_assistant.lower()
            )

            # ✅ ключевой фикс: продолжение диалога
            if waiting_for_vacancy_text:
                if ".pdf" in query.lower().strip():
                    return agent.analyze_vacancy_pdf_tool(query.strip(), history=history)
                return agent.analyze_vacancy_text_tool(query, history=history)

            # обычный single-turn
            if "ваканси" in q or "red flag" in q or "green flag" in q:
                if ".pdf" in q:
                    return agent.analyze_vacancy_pdf_tool(query.strip(), history=history)
                return "Вставь сюда текст вакансии целиком (или пришли путь к PDF), и я сделаю юридический разбор red/green flags."

            if "вопрос" in q or "собесед" in q or "интервью" in q:
                return agent.generate_interview_questions_tool(position=query, level="middle", history=history)

            return agent.chat(query, history=history)



    def _purpose_message(self) -> str:
        return (
            "Я специализированный ассистент по карьерной стратегии/подготовке и устройству.\n\n"
            "Могу:\n"
            "• составлять roadmap развития,\n"
            "• анализировать резюме и улучшать его,\n"
            "• писать сопроводительные письма,\n"
            "• анализировать вакансии на red/green flags,\n"
            "• генерировать вопросы к собеседованию.\n"
        )

    def process_query(self, user_query: str, resume_file_path: Optional[str] = None) -> str:
        q = user_query.lower()

        # фильтр “не по теме” можешь оставить, но я делаю проще и мягче
        base_keywords = [
            "резюме", "cv", "roadmap", "карьер", "сопровод", "письмо",
            "ваканси", "собесед", "интервью", "вопрос", "red flag", "green flag"
        ]


        # спец-кейс: анализ резюме по PDF
        resume_keywords = ['резюме', 'cv']
        analyze_keywords = ['анализ', 'проанализируй', 'оцен', 'разбор']

        is_resume_query = any(k in q for k in resume_keywords)
        is_analyze_request = any(k in q for k in analyze_keywords)

        if is_resume_query and is_analyze_request and resume_file_path is not None:
            # ✅ было self.career_agent
            response = self.career_preparation_agent.analyze_resume_tool(resume_file_path)
            self._append_history(user_query, response)
            return response

        initial_state = AgentState(
            messages=[],
            current_agent="orchestrator",
            user_query=user_query,
        )
        result = self.graph.invoke(initial_state)
        response = result.get("final_response", "Не удалось обработать запрос") if isinstance(result, dict) else getattr(result, "final_response", None)
        response = response or "Не удалось обработать запрос"

        self._append_history(user_query, response)
        return response


In [9]:
system = MultiAgentSystem()

while True:
    user_text = input("Ты: ")
    if user_text.strip().lower() in {"выход", "exit", "quit"}:
        break

    resume_path = None
    q = user_text.lower()
    if "резюме" in q and "анализ" in q:
        resume_path = input("📂 Введи путь к PDF с резюме: ").strip()

    answer = system.process_query(user_text, resume_file_path=resume_path)
    print("Бот:", answer)
    print("-" * 80)

Ты: составь roadmap по изучению продуктовой аналитики
Бот: Чтобы составить дорожную карту, пришли 3 строки:
profession: ...
goal: ...
background: ...

--------------------------------------------------------------------------------
Ты: продуктовый аналитик, я ds специалист, хочу овладеть аналитикой. Работаю middle ds
Бот: ### Roadmap по освоению продуктовой аналитики (Roadmap изучения продуктовой аналитики)

#### Profession: Продуктовый аналитик  
**Goal:** Овладение всеми необходимыми компетенциями продуктового аналитика и интеграция знаний в текущую роль Middle DS специалиста.  
**Background:** Есть опыт работы Data Scientist среднего уровня, хорошо владею Python, SQL, умею строить модели машинного обучения, работаю с инструментами BI и визуализации (Power BI, Tableau). Имеются знания основ продуктовой аналитики, однако нужно углубиться и освоить специфические навыки.

---

## 1. Базовые знания продуктовой аналитики  
### Неделя 1–4  
- **Суть продуктовой аналитики:** Определение цел


KeyboardInterrupt

